In [1]:
! pip install geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv

/bin/bash: /media/mutakabbir/HDD_2TB_02/Forest_Fire/.venv/bin/pip: /media/mutakabbir/HDD_2TB_01/Forest_Fire/.venv/bin/python: bad interpreter: No such file or directory


In [2]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

import os
from dotenv import load_dotenv

In [3]:
CWEEDS_META_CSV_FLIE_PATH = "../../data/ics_meta/CWEEDS_2020_stns_all_REV_20210324.xlsx"
PATH_TO_DOT_ENV = "../../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

CWEEDS_META_TABLE_NAME = "W_m"

In [4]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

In [5]:
engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

In [6]:
# read the dataset
cweeds_meta_df = pd.read_excel(
    CWEEDS_META_CSV_FLIE_PATH, 
    index_col=4
    )
cweeds_meta_df.head()

,version,name,prov,country,lat,lon,UTC_offset,elev(m),first_yr,last_yr
climate_ID,,,,,,,,,,
3010010,CWEEDS2011,ABEE AGDM,AB,CAN,54.28,-112.97,-7,664.0,2003,2017
3010237,CWEEDS2011,ANDREW AGDM,AB,CAN,53.92,-112.28,-7,625.0,2003,2017
3060406,CWEEDS2011,ATMORE AGDM,AB,CAN,54.78,-112.82,-7,574.0,2003,2017
3050519,CWEEDS2011,BANFF CS,AB,CAN,51.19,-115.55,-7,1396.9,1998,2017
3030525,CWEEDS2011,BARNWELL AGDM,AB,CAN,49.80,-112.30,-7,824.2,2003,2017


In [7]:
# convert to Geodataframe
cweeds_meta_gdf = gpd.GeoDataFrame(
    cweeds_meta_df,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(
        cweeds_meta_df.lon, 
        cweeds_meta_df.lat
    ),
)

In [8]:
try:
    cweeds_meta_gdf.to_postgis(name=CWEEDS_META_TABLE_NAME, con=engine, if_exists='replace', index=True)  
except Exception as e:
    print(e)